In [1]:
from natasha import LocationExtractor
from collections import defaultdict
import re
from datetime import datetime
from itertools import islice
import tqdm

In [2]:
def Make_matches(text):
    ne = LocationExtractor()
    matches = ne(text)
    return matches

In [21]:
def file_walker(file):
    rows = []
    not_proper = []
    with open(file, 'r', encoding='utf-8') as f:
        train = list(islice(f, 25000))
    for line in train:
        line = line.strip('\n')
        sets = line.split(',#')
        for setr in range(len(sets)):
            sets[setr] = sets[setr].strip(',#')
        if len(sets) == 10:
            rows.append(sets)
        else:
            not_proper.append(sets)
    return rows, not_proper

In [22]:
notes, not_proper = file_walker('notes_beautiful_all.csv')

In [23]:
def Extract_places(notes, all_names):
    
    for note in tqdm.tqdm_notebook(notes[1:]):            
        cases = defaultdict(int)
        note_names = []
        try:
            matches = Make_matches(note[2])
        except IndexError:
            print(note)
        
        for match in matches:
            name = match.fact
            string = ''
            if name != None:
                string += str(name) + ' '
            note_names.append(string[:-1])
        
        for name in note_names:
            cases[name] += 1
            
        if len(note) >= 4:
            arr = [note[0], note[1], note[3]]
        else:
            arr = [note[0], note[1], "Not known"]
        tups = []
        
        for key in cases:
            place = re.findall("name='(.+?)'", key)[0]
            tups.append((place, cases[key]))
        arr.append(tups)
        all_names.append(arr)
    return(all_names)

In [24]:
start_time = datetime.now()
all_names = []
names_all = Extract_places(notes, all_names)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))


Duration: 1:29:53.760924


In [25]:
len(names_all)

24999

In [26]:
def file_walker(file):
    rows = []
    with open(file, 'r', encoding='utf-8') as f:
        train = f.readlines()
    for line in train:
        line = line.strip('\n')
        sets = line.split(',#')
        for setr in range(len(sets)):
            sets[setr] = sets[setr].strip(',#')
        rows.append(sets)
    return rows

In [27]:
diary = file_walker('diary_beautiful.csv')
diaryid_persid = {}
for diar in diary[1:]:
    diaryid_persid[diar[0]] = diar[1]

In [33]:
final_data = [['note_id', 'diary_id', 'author_id', 'note_date', 'places']]
for name in names_all:
    if name[1] != '"2"': # почему-то в файле нет id == 2
        final_data.append([name[0], name[1], diaryid_persid[name[1]], name[2], name[3]])
    else:
        final_data.append([name[0], "2", "Not known", name[2], name[3]])

In [19]:
final_data[5]

['"39085"',
 '"234"',
 '"241"',
 '"1933-01-15"',
 [('калинин', 1),
  ('нея', 1),
  ('то', 1),
  ('красный', 1),
  ('ленинград', 1),
  ('реж', 1),
  ('кавказ', 1),
  ('москва', 3),
  ('сена', 2)]]

In [34]:
start_time = datetime.now()
with open("places.csv", 'w', encoding='utf-8') as f:
    for header in final_data[0]:
        if header != final_data[0][-1]:
            f.write(str(header)+',#')
        if header == final_data[0][-1]:
            f.write(str(header)+'\n')
    for data in final_data[1:]:
        if len(data[-1]) != 0:
            for content in data:
                if content != data[-1]:
                    f.write(str(content)+',#')
                if content == data[-1]:
                    f.write('"')
                    for tup in content:
                        if tup != content[-1]:
                            f.write(str(tup[0])+'->'+str(tup[1])+',')
                        else:
                            f.write(str(tup[0])+'->'+str(tup[1])+'"\n')
        else:
            for content in data:
                if content != data[-1]:
                    f.write(str(content)+',#')
                if content == data[-1]:
                    f.write('\n')
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:00.173691
